In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve

In [3]:
train_data = pd.read_csv("worked_data_train.csv", index_col = 0)
Test = pd.read_csv("worked_data_test.csv", index_col = 0)

In [4]:
for feature in train_data.select_dtypes("object").columns:
    train_data[feature].fillna(train_data[feature].mode()[0], inplace = True)

In [5]:
train_data = pd.get_dummies(train_data)

In [6]:
for feature in train_data.select_dtypes("float64").columns:
    train_data[feature].fillna(train_data[feature].mean(), inplace = True)
    
for feature in train_data.select_dtypes("int64").columns:
    train_data[feature].fillna(train_data[feature].median(), inplace = True)

In [7]:
Test =  pd.get_dummies(Test)
for feature in Test.select_dtypes("float64").columns:
    Test[feature].fillna(train_data[feature].mean(), inplace = True)
    
for feature in Test.select_dtypes("int64").columns:
    Test[feature].fillna(train_data[feature].median(), inplace = True)

In [8]:
train_data.shape

(307511, 453)

In [9]:
Test.shape

(48744, 449)

In [10]:
def fun(y,x):
    for i in x:
        if i not in y:
            print(i)

In [11]:
fun(Test,train_data)

TARGET
CODE_GENDER_XNA
NAME_INCOME_TYPE_Maternity leave
NAME_FAMILY_STATUS_Unknown


In [12]:
train_data = train_data.drop(['CODE_GENDER_XNA', 'NAME_INCOME_TYPE_Maternity leave', 'NAME_FAMILY_STATUS_Unknown'], 1)

In [13]:
train_data.shape

(307511, 450)

In [14]:
train_data.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002.0,1.0,0.0,202500.0,406597.5,24700.5,351000.0,0.018801,9461.0,637.0,...,0,0,0,0,0,0,1,0,1,0
1,100003.0,0.0,0.0,270000.0,1293502.5,35698.5,1129500.0,0.003541,16765.0,1188.0,...,0,1,0,0,0,0,0,0,1,0
2,100004.0,0.0,0.0,67500.0,135000.0,6750.0,135000.0,0.010032,19046.0,225.0,...,0,0,0,0,0,1,0,0,1,0
3,100006.0,0.0,0.0,135000.0,312682.5,29686.5,297000.0,0.008019,19005.0,3039.0,...,0,0,0,0,0,1,0,0,1,0
4,100007.0,0.0,0.0,121500.0,513000.0,21865.5,513000.0,0.028663,19932.0,3038.0,...,0,0,0,0,0,1,0,0,1,0


In [14]:
k_fold = KFold(n_splits = 10, random_state = 42, shuffle = True)
folds = []
for train_index, test_index in k_fold.split(train_data):
    to_append = {"Train" : train_data.iloc[train_index,], "Test" : train_data.iloc[test_index]}
    folds.append(to_append)

In [16]:
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

## Logistic Regression

In [66]:
roc_scores = []
for fold in folds:
    log_reg = LogisticRegression(C = 0.0001)
    log_reg.fit(fold["Train"].drop(["TARGET"], axis = 1), fold["Train"]["TARGET"])
    probabilities = log_reg.predict_proba(fold["Test"].drop(["TARGET"], axis = 1))
    roc_scores.append(roc_auc_score(fold["Test"]["TARGET"], probabilities[:, 1]))
    print("Hey")

Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey


In [67]:
sum(roc_scores) / len(roc_scores)

0.6166448087998891

In [15]:
Test_or= pd.read_csv("application_test.csv")

In [69]:
result = Test_or['SK_ID_CURR']
P = log_reg.predict_proba(Test)
preds = pd.DataFrame()
preds['SK_ID_CURR'] = result
l = []
for i in P:
    l.append(i[1])
preds['TARGET'] = l
preds.to_csv('logres.csv', index = False)

## LGBM

In [73]:
import lightgbm

In [36]:
Test = Test.drop([])

,SK_ID_CURR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100001.0,0.0,135000.0,568800.0,20560.5,450000.0,0.018850,19241.0,2329.000000,-5170.0,...,0,0,0,0,0,0,1,0,1,0
1,100005.0,0.0,99000.0,222768.0,17370.0,180000.0,0.035792,18064.0,4469.000000,-9118.0,...,0,0,0,0,0,0,0,0,0,0
2,100013.0,0.0,202500.0,663264.0,69777.0,630000.0,0.019101,20038.0,4458.000000,-2175.0,...,0,0,0,0,0,0,0,0,0,0
3,100028.0,2.0,315000.0,1575000.0,49018.5,1575000.0,0.026392,13976.0,1866.000000,-2000.0,...,0,0,0,0,0,1,0,0,1,0
4,100042.0,0.0,270000.0,959688.0,34600.5,810000.0,0.025164,18604.0,12009.000000,-6116.0,...,0,1,0,0,0,0,0,0,1,0
5,100057.0,2.0,180000.0,499221.0,22117.5,373500.0,0.022800,16685.0,2580.000000,-10125.0,...,0,0,0,0,0,0,0,0,0,0
6,100065.0,0.0,166500.0,180000.0,14220.0,180000.0,0.005144,9516.0,1387.000000,-5063.0,...,0,0,0,0,0,0,0,0,0,0
7,100066.0,0.0,315000.0,364896.0,28957.5,315000.0,0.046220,12744.0,1013.000000,-1686.0,...,0,0,0,0,0,0,1,0,1,0
8,100067.0,1.0,162000.0,45000.0,5337.0,45000.0,0.018634,10395.0,2625.000000,-8124.0,...,0,0,0,0,0,0,0,0,0,0
9,100074.0,0.0,67500.0,675000.0,25447.5,675000.0,0.003122,23670.0,2463.633092,-7490.0,...,0,0,0,0,0,0,1,0,1,0


In [85]:
roc_scores_lgbm = []
for fold in folds:
    clf = LGBMClassifier()
    clf.fit(fold["Train"].drop(["TARGET"], axis = 1), fold["Train"]["TARGET"])
    probabilities = clf.predict_proba(fold["Test"].drop(["TARGET"], axis = 1))
    roc_scores_lgbm.append(roc_auc_score(fold["Test"]["TARGET"], probabilities[:, 1]))
    print("Hey")

LightGBMError: Do not support special JSON characters in feature name.

## XGBoost

In [16]:
from xgboost import XGBClassifier

In [17]:
roc_scores_xg = []
for fold in folds:
    xg = XGBClassifier()
    xg.fit(fold["Train"].drop(["TARGET"], axis = 1), fold["Train"]["TARGET"])
    probabilities = xg.predict_proba(fold["Test"].drop(["TARGET"], axis = 1))
    roc_scores_xg.append(roc_auc_score(fold["Test"]["TARGET"], probabilities[:, 1]))
    print("Hey")

Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey
Hey


In [18]:
sum(roc_scores_xg) / len(roc_scores_xg)

0.7658449900783483

In [19]:
result_xg = Test_or['SK_ID_CURR']
P = xg.predict_proba(Test)
preds = pd.DataFrame()
preds['SK_ID_CURR'] = result_xg
l = []
for i in P:
    l.append(i[1])
preds['TARGET'] = l
preds.to_csv('xg2.csv', index = False)